In [95]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from collections import Counter

In [96]:
with open('../data/preprocessed_X_visit_over3.pkl', 'rb') as f:
    data_x = pickle.load(f)

with open('../data/preprocessed_y_visit_over3.pkl', 'rb') as f: 
    labels = pickle.load(f)

In [97]:
visit_gt2t_new = {}
for k, v in data_x.items():
    # print(k, v)
    visit_new = {}
    for v_k, v_v in v.items():
        total_list = list(set(v_v['diagnoses'])) + list(set(v_v['procedures'])) + v_v['drugs']
        # visit_new[v_k] = v_v
        if len(total_list) > 1:
            visit_new[v_k] = {'diagnoses': None, 'procedures':None, 'drugs': None, 'admitdate':None}
            visit_new[v_k]['diagnoses'] = list(set(v_v['diagnoses'])) 
            visit_new[v_k]['procedures'] = list(set(v_v['procedures'])) 
            visit_new[v_k]['drugs'] = list(set(v_v['drugs'])) 
            visit_new[v_k]['admitdate'] = v_v['admitdate']
    visit_gt2t_new[k] = visit_new

In [98]:
adm = pd.read_csv('./admissions.csv.gz', compression='gzip', usecols=['subject_id', 'hadm_id', 'admittime', 'admission_type', 'race'])
patients = pd.read_csv('./patients.csv')
patients = patients[['subject_id', 'gender','anchor_age','anchor_year']]
patients['yob']= patients['anchor_year'] - patients['anchor_age']

In [99]:
visit_gt2t_last = {}
for k, v in visit_gt2t_new.items():
    # break
    if len(v) >= 2:
        visit_gt2t_last[k] = v

In [150]:
d_list = list()
p_list = list()
dr_list = list()
dr_length = list()
for k, visit in visit_gt2t_new.items():
    for v_id, v in visit.items():
        d_list.extend(v['diagnoses'])
        p_list.extend(v['procedures'])
        dr_list.extend(v['drugs'])
        dr_length.append(len(v['drugs']))
        
# d_list = list(set(d_list))
# p_list = list(set(p_list))
# dr_list = list(set(dr_list))

In [151]:
max(dr_length)

276

In [152]:
drug_dict = dict(Counter(dr_list))
sorted_drug_dict = dict(sorted(drug_dict.items(), key=lambda item: item[1], reverse=False))

In [153]:
d_list = list(set(d_list))
p_list = list(set(p_list))
dr_list = list(set(dr_list))

In [154]:
len(d_list), len(p_list), len(dr_list)

(869, 747, 3528)

In [157]:
data_dict = {}
for p_id, v_data in tqdm(visit_gt2t_last.items()):
    visit_data = {'seq': [], 'data':[], 'time': [], #'total_seq': ['[CLS]'], 'seq_seg': [0], 'code_type': [0],
                  'gender': patients.loc[patients['subject_id'] == p_id, 'gender'].unique()[0],
                  'race': adm.loc[adm['subject_id'] == p_id, 'race'].unique()[0]}
    seg_idx = 0
    for v_id, record in v_data.items():
        record['visit_id'] = v_id
        record['admission_type'] = adm.loc[(adm['subject_id'] == p_id) & (adm['hadm_id'] == v_id), 'admission_type'].values[0]
        dt_obj = datetime.strptime(record['admitdate'], '%Y-%m-%d')
        age_at_adm = dt_obj.year - patients.loc[patients['subject_id'] == p_id, 'yob'].values[0]
        if age_at_adm < 0:
            age_at_adm = 90
        record['age'] = age_at_adm
        if len(record['drugs']) > 50:
            sorted_drugs = sorted(sorted_drug_dict, key=sorted_drug_dict.get, reverse=True)
            top100_drugs = [elem for elem in sorted_drugs if elem in record['drugs']][:50]
            seq_data = record['diagnoses'] + record['procedures'] + top100_drugs
        else:
            seq_data = record['diagnoses'] + record['procedures'] + record['drugs']
        
        # pretrained_seq = seq_data + ['[SEP]']
        # pretrained_seq_seg = [seg_idx] * len(seq_data)
        # pretrained_code_type = [1] * len(record['diagnoses']) + [2] * len(record['procedures']) + [3] * len(record['drugs']) + [0]
        # visit_data['total_seq'].extend(pretrained_seq)
        # visit_data['seq_seg'].extend(pretrained_seq_seg)
        # visit_data['code_type'].extend(pretrained_code_type)
        visit_data['seq'].append(seq_data)
        visit_data['data'].append(record)
        visit_data['time'].append(dt_obj)
        # seg_idx += 1
    visit_data['seq'].append(['global'])
    visit_data['time'].append(visit_data['time'][-1])
    
    td_list = [(visit_data['time'][-1]- dt).days for dt in visit_data['time']]
    visit_data['timedelta'] = td_list
    visit_data['labels_origin'] = labels[p_id]['top100_label']
    visit_data['labels_bin'] = labels[p_id]['top100_label_bin']
    data_dict[p_id] = visit_data

100%|██████████| 43096/43096 [05:48<00:00, 123.60it/s]


In [158]:
with open('../data/data_dict_pre_drug_filter.pkl', 'wb') as f:
    pickle.dump(data_dict, f)

In [159]:
with open('../data/data_dict_pre_drug_filter.pkl', 'rb') as f:
    data_dict = pickle.load(f)

In [160]:
len(d_list), len(p_list), len(dr_list)

(869, 747, 3528)

In [161]:
code2idx = {'padding':0}

idx = 1
for d in d_list:
    code2idx[d] = idx
    idx += 1

for p in p_list:
    code2idx[p] = idx
    idx += 1

dr_idx = 1
for dr in dr_list:  
    code2idx[dr] = idx
    idx += 1

code2idx['global'] = idx
code2idx['[CLS]'] = idx + 1
code2idx['[SEP]'] = idx + 2
code2idx['[MASK]'] = idx + 3

In [162]:
len(code2idx)

5149

In [163]:
code2idx['[MASK]']

5148

In [164]:
with open('../data/code_indices/code2idx_pre.pkl', 'wb') as f:
    pickle.dump(code2idx, f)

In [165]:
with open('../data/code_indices/code2idx_pre.pkl', 'rb') as f:
    code2idx = pickle.load(f)

In [166]:
data_dict_idx = data_dict.copy()

In [167]:
for k, visit in tqdm(data_dict_idx.items()):
    visit['seq_idx'] = [list(map(code2idx.get, v)) for v in visit['seq']]

100%|██████████| 43096/43096 [00:00<00:00, 51789.93it/s]


In [168]:
# for k, visit in tqdm(data_dict_idx.items()):
#     visit['total_seq_idx'] = list(map(code2idx.get, visit['total_seq']))

In [169]:
data_dict_new= dict()
for p_id in tqdm(list(data_dict_idx.keys())):
    p_data = {'visit':[], 'visit_idx': [], 'visit_length':0, \
              'seq':None, 'seq_idx':None, 'timedelta': None, 'label':[]}
    
    for v_id, v_data in enumerate(data_dict_idx[p_id]['data']):
        p_data['visit'].append(v_data['visit_id'])
        p_data['visit_idx'].append(v_id)
        p_data['seq'] = data_dict_idx[p_id]['seq']
        p_data['seq_idx'] = data_dict_idx[p_id]['seq_idx']
        p_data['time'] = data_dict_idx[p_id]['time']
        p_data['timedelta'] = data_dict_idx[p_id]['timedelta']
    
    p_data['label'] = np.squeeze(data_dict_idx[p_id]['labels_bin'])
    data_dict_new[p_id] = p_data

100%|██████████| 43096/43096 [00:00<00:00, 114914.78it/s]


In [170]:
data_dict_max50 = dict()
for p_id , data in data_dict_new.items():
    new_data = dict()
    if len(data['visit']) > 50:
        print(p_id, len(data['visit']))
    new_data['visit'] = data['visit'][-50:]
    new_data['visit_idx'] = data['visit_idx'][-50:]
    new_data['visit_length'] = len(data['visit'][-50:])
    new_data['seq'] = data['seq'][-51:-1]
    new_data['seq_idx'] = data['seq_idx'][-51:-1]
    new_data['time'] = data['time'][-51:-1]
    new_data['timedelta'] = data['timedelta'][-51:-1]
    assert len(data['visit_idx'][-50:]) == len(data['seq'][-51:-1])
    new_data['label'] = data['label']
    data_dict_max50[p_id] = new_data

10123949 55
10264646 76
10577647 78
10578325 67
10580201 59
10714009 113
10913302 64
11021643 63
11296936 76
11413236 77
11553072 69
11582633 81
11714071 62
11761621 56
11818101 64
11888614 56
11890447 66
11965254 90
12251785 79
12468016 88
12547294 69
12563258 60
12596559 63
13166511 53
13297743 91
13470788 52
13475033 96
13813803 68
13877234 60
13999829 54
14029474 52
14318651 81
14394983 124
15084163 53
15107347 51
15108590 52
15114531 67
15229574 79
15464144 75
15496609 133
15935768 56
16124481 62
16233333 67
16439884 58
16615356 55
16662316 129
16675371 66
16809525 57
16924675 54
17011846 56
17051420 54
17204468 59
17340686 51
17477304 51
17517983 94
17716210 80
17937834 61
18001923 69
18136887 62
18284271 82
18376342 52
18553055 55
18655830 69
18656167 55
18676703 59
18902344 70
18970086 68
19127408 51
19133405 78
19610016 52
19713100 62
19759225 63
19921471 57


In [171]:
def convert_code_type(code):
    if code.startswith('d'):
        return 1
    elif code.startswith('pcs'):
        return 2
    elif code.startswith('p'):
        return 3
    elif code in ['[CLS]', '[SEP]']:
        return 0

In [172]:
tota1_len_list = []
for p_id , data in data_dict_max50.items():
    total_seq = ['[CLS]'] + [item for sublist in data['seq'] for item in sublist + ['[SEP]']]
    total_seq_idx = [code2idx['[CLS]']] + [item for sublist in data['seq_idx'] for item in sublist + [code2idx['[SEP]']]]
    total_code_type = list(map(convert_code_type, total_seq))
    total_vis_seg = [0] + [elem for idx, sublist in enumerate(data['seq_idx']) for elem in [idx] * len(sublist) + [idx]]
    assert len(total_seq) == len(total_seq_idx) == len(total_code_type) == len(total_vis_seg)
    data['total_seq'] = total_seq
    data['total_seq_idx'] = total_seq_idx
    data['code_type'] = total_code_type
    data['visit_seg'] = total_vis_seg
    tota1_len_list.append(len(total_seq))

In [173]:
with open('../data/data_dict_pretrained_maxlen50_filter_drugs.pkl', 'wb') as f:
    pickle.dump(data_dict_max50, f)

In [174]:
with open('../data/data_dict_pretrained_maxlen50_filter_drugs.pkl', 'rb') as f:
    data_dict_max50 = pickle.load(f)

In [195]:
data_dict_max50_slide = dict()
for p_id , data in tqdm(data_dict_max50.items()):
    slide_length = 5
    if len(data['visit']) > slide_length:
        for i in range(len(data['visit']) - slide_length + 1):
            visit_data = dict()
            visit_data['visit'] = data['visit'][i:i+slide_length]
            visit_data['visit_idx'] = data['visit_idx'][i:i+slide_length]
            visit_data['visit_length'] = len(data['visit'][i:i+slide_length])
            
            slide_seq = data['seq'][i:i+slide_length]
            slide_seq_idx = data['seq_idx'][i:i+slide_length]
            
            total_seq = ['[CLS]'] + [item for sublist in slide_seq for item in sublist + ['[SEP]']]
            total_seq_idx = [code2idx['[CLS]']] + [item for sublist in slide_seq_idx for item in sublist + [code2idx['[SEP]']]]
            total_code_type = list(map(convert_code_type, total_seq))
            total_vis_seg = [visit_data['visit_idx'][0]] + [elem for idx, v_idx in enumerate(visit_data['visit_idx']) for elem in [v_idx] * len(slide_seq_idx[idx]) + [v_idx]]
            assert len(total_seq) == len(total_seq_idx) == len(total_code_type) == len(total_vis_seg)
            
            visit_data['total_seq'] = total_seq
            visit_data['total_seq_idx'] = total_seq_idx
            visit_data['code_type'] = total_code_type
            visit_data['visit_seg'] = total_vis_seg
            visit_data['total_len'] = len(total_seq)
            visit_data['time'] = data['time'][i:i+slide_length]

            timedelta_replacement = {visit_data['visit_idx'][idx]: td for idx, td in enumerate(data['timedelta'][i:i+slide_length])}
            visit_data['timedelta'] = [timedelta_replacement[val] for val in total_vis_seg]
            timevec_replacement = {visit_data['visit_idx'][idx]: [tm.year, tm.month, tm.day, tm.isocalendar().week] \
                                   for idx, tm in enumerate(data['time'][i:i+slide_length])}
            visit_data['timevec'] = [timevec_replacement[val] for val in total_vis_seg]
            data_dict_max50_slide[f'{str(p_id)}_slide{str(i)}'] = visit_data
    else:
        visit_data = dict()
        visit_data['visit'] = data['visit']
        visit_data['visit_idx'] = data['visit_idx']
        visit_data['visit_length'] = data['visit_length']
        
        total_seq = ['[CLS]'] + [item for sublist in data['seq'] for item in sublist + ['[SEP]']]
        total_seq_idx = [code2idx['[CLS]']] + [item for sublist in data['seq_idx'] for item in sublist + [code2idx['[SEP]']]]
        total_code_type = list(map(convert_code_type, total_seq))
        total_vis_seg = [data['visit_idx'][0]] + [elem for idx, v_idx in enumerate(data['visit_idx']) for elem in [v_idx] * len(data['seq_idx'][idx]) + [v_idx]]
        assert len(total_seq) == len(total_seq_idx) == len(total_code_type) == len(total_vis_seg)
        
        visit_data['total_seq'] = total_seq
        visit_data['total_seq_idx'] = total_seq_idx
        visit_data['code_type'] = total_code_type
        visit_data['visit_seg'] = total_vis_seg
        visit_data['total_len'] = len(total_seq)
        visit_data['time'] = data['time']
        
        timedelta_replacement = {visit_data['visit_idx'][idx]: td for idx, td in enumerate(data['timedelta'])}
        visit_data['timedelta'] = [timedelta_replacement[val] for val in total_vis_seg]
        timevec_replacement = {visit_data['visit_idx'][idx]: [tm.year, tm.month, tm.day, tm.isocalendar().week] \
                               for idx, tm in enumerate(data['time'])}
        visit_data['timevec'] = [timevec_replacement[val] for val in total_vis_seg]
        data_dict_max50_slide[f'{str(p_id)}_slide{str(1)}'] = visit_data

100%|██████████| 43096/43096 [00:18<00:00, 2305.88it/s]


In [196]:
len(data_dict_max50_slide[f'{str(p_id)}_slide{str(1)}']['total_seq'])

171

In [197]:
len(data_dict_max50_slide[f'{str(p_id)}_slide{str(i)}']['total_seq'])

178

In [198]:
f'{str(p_id)}_slide{str(i)}'

'19999784_slide12'

In [199]:
len(visit_data['total_seq_idx'])

178

In [200]:
len(visit_data['timevec'] )

178

In [201]:
total_len_list = []
for p_id in list(data_dict_max50_slide.keys()):
    total_len_list.append(data_dict_max50_slide[p_id]['total_len'])

In [202]:
np.min(total_len_list)

7

In [203]:
np.max(total_len_list)

407

In [204]:
data_dict_max50_slide['11052252_slide1']['total_len']

113

In [205]:
with open('../data/data_dict_pretrained_maxlen50_slide_filter_drugs.pkl', 'wb') as f:
    pickle.dump(data_dict_max50_slide, f)

In [207]:
data['visit'][-5:]

[27319264, 29324445, 23406899, 29889147, 29956342]

In [209]:
data_dict_max50_ft = dict()
for p_id , data in tqdm(data_dict_max50.items()):
    max_length = 5
    if len(data['visit']) > slide_length:
        visit_data = dict()
        visit_data['visit'] = data['visit'][-5:]
        visit_data['visit_idx'] = data['visit_idx'][-5:]
        visit_data['visit_length'] = len(data['visit'][-5:])
        
        slide_seq = data['seq'][-5:]
        slide_seq_idx = data['seq_idx'][-5:]
        
        total_seq = ['[CLS]'] + [item for sublist in slide_seq for item in sublist + ['[SEP]']]
        total_seq_idx = [code2idx['[CLS]']] + [item for sublist in slide_seq_idx for item in sublist + [code2idx['[SEP]']]]
        total_code_type = list(map(convert_code_type, total_seq))
        total_vis_seg = [visit_data['visit_idx'][0]] + [elem for idx, v_idx in enumerate(visit_data['visit_idx']) for elem in [v_idx] * len(slide_seq_idx[idx]) + [v_idx]]
        assert len(total_seq) == len(total_seq_idx) == len(total_code_type) == len(total_vis_seg)
        
        visit_data['total_seq'] = total_seq
        visit_data['total_seq_idx'] = total_seq_idx
        visit_data['code_type'] = total_code_type
        visit_data['visit_seg'] = total_vis_seg
        visit_data['total_len'] = len(total_seq)
        visit_data['time'] = data['time'][-5:]

        timedelta_replacement = {visit_data['visit_idx'][idx]: td for idx, td in enumerate(data['timedelta'][-5:])}
        visit_data['timedelta'] = [timedelta_replacement[val] for val in total_vis_seg]
        timevec_replacement = {visit_data['visit_idx'][idx]: [tm.year, tm.month, tm.day, tm.isocalendar().week] \
                                for idx, tm in enumerate(data['time'][-5:])}
        visit_data['timevec'] = [timevec_replacement[val] for val in total_vis_seg]
        data_dict_max50_ft[p_id] = visit_data
    else:
        visit_data = dict()
        visit_data['visit'] = data['visit']
        visit_data['visit_idx'] = data['visit_idx']
        visit_data['visit_length'] = data['visit_length']
        
        total_seq = ['[CLS]'] + [item for sublist in data['seq'] for item in sublist + ['[SEP]']]
        total_seq_idx = [code2idx['[CLS]']] + [item for sublist in data['seq_idx'] for item in sublist + [code2idx['[SEP]']]]
        total_code_type = list(map(convert_code_type, total_seq))
        total_vis_seg = [data['visit_idx'][0]] + [elem for idx, v_idx in enumerate(data['visit_idx']) for elem in [v_idx] * len(data['seq_idx'][idx]) + [v_idx]]
        assert len(total_seq) == len(total_seq_idx) == len(total_code_type) == len(total_vis_seg)
        
        visit_data['total_seq'] = total_seq
        visit_data['total_seq_idx'] = total_seq_idx
        visit_data['code_type'] = total_code_type
        visit_data['visit_seg'] = total_vis_seg
        visit_data['total_len'] = len(total_seq)
        visit_data['time'] = data['time']
        
        timedelta_replacement = {visit_data['visit_idx'][idx]: td for idx, td in enumerate(data['timedelta'])}
        visit_data['timedelta'] = [timedelta_replacement[val] for val in total_vis_seg]
        timevec_replacement = {visit_data['visit_idx'][idx]: [tm.year, tm.month, tm.day, tm.isocalendar().week] \
                               for idx, tm in enumerate(data['time'])}
        visit_data['timevec'] = [timevec_replacement[val] for val in total_vis_seg]
        data_dict_max50_ft[p_id] = visit_data

100%|██████████| 43096/43096 [00:03<00:00, 12404.05it/s]


In [211]:
with open('../data/data_dict_finetuning_maxlen50_slide_filter_drugs.pkl', 'wb') as f:
    pickle.dump(data_dict_max50_ft, f)

In [200]:
data_dict_max50_slide.keys()

dict_keys(['10000032_slide1', '10000826_slide1', '10000935_slide0', '10000935_slide1', '10000980_slide0', '10000980_slide1', '10000980_slide2', '10000980_slide3', '10001186_slide1', '10001319_slide1', '10001338_slide1', '10001401_slide0', '10001401_slide1', '10001401_slide2', '10001884_slide0', '10001884_slide1', '10001884_slide2', '10001884_slide3', '10001884_slide4', '10001884_slide5', '10001884_slide6', '10001884_slide7', '10001884_slide8', '10001884_slide9', '10001884_slide10', '10001884_slide11', '10001884_slide12', '10001884_slide13', '10001884_slide14', '10001884_slide15', '10001884_slide16', '10001884_slide17', '10001884_slide18', '10002013_slide0', '10002013_slide1', '10002013_slide2', '10002013_slide3', '10002013_slide4', '10002013_slide5', '10002013_slide6', '10002013_slide7', '10002013_slide8', '10002131_slide1', '10002155_slide0', '10002155_slide1', '10002221_slide0', '10002221_slide1', '10002428_slide0', '10002428_slide1', '10002428_slide2', '10002428_slide3', '10002430_s

In [135]:
# Given dictionary and list
count_dict = {'a': 10, 'b': 1, 'c': 5, 'e': 3, 'f': 5, 'g': 2, 'h': 12}
data_list = ['a', 'b', 'c', 'a', 'g', 'e', 'f', 'a', 'b', 'c', 'd']

# Sort elements by frequency in descending order and select top 3 elements that are in data_list
sorted_elements = sorted(count_dict, key=count_dict.get, reverse=True)
top_elements = [elem for elem in sorted_elements if elem in data_list][:3]

# Filter data_list to include only elements in the adjusted top 3 frequency list
filtered_list = [item for item in data_list if item in top_elements]
filtered_list


['a', 'c', 'a', 'f', 'a', 'c']

In [136]:
sorted_elements

['h', 'a', 'c', 'f', 'e', 'g', 'b']

In [137]:
top_elements

['a', 'c', 'f']

In [138]:
sorted_elements

['h', 'a', 'c', 'f', 'e', 'g', 'b']

In [139]:
sorted_drugs = sorted(sorted_drug_dict, key=sorted_drug_dict.get, reverse=True)

In [143]:
sorted_drug_dict['p_HEPA5I']

118607

In [140]:
sorted_drugs[:100]

['p_NACLFLUSH',
 'p_HEPA5I',
 'p_DOCU100',
 'p_ACET325',
 'p_SENN187',
 'p_ONDAN4I',
 'p_DEX50SY',
 'p_OXYC5',
 'p_MAG2PM',
 'p_GLUC1I',
 'p_ASA81',
 'p_OMEP20',
 'p_APAP500',
 'p_BAG50',
 'p_INFL0.5LF',
 'p_MICROK10',
 'p_PANT40',
 'p_NS1000',
 'p_BISA10R',
 'p_GLYC17',
 'p_GLUCOSEGEL',
 'p_INSULIN',
 'p_MVI',
 'p_NSLV',
 'p_HEPA10SYR',
 'p_KCL20P',
 'p_CALC500',
 'p_VANC1F',
 'p_VANCOBASE',
 'p_NS500',
 'p_BISA5',
 'p_ALBU3H',
 'p_LORA5',
 'p_HUMU100I',
 'p_HYDR1SYR',
 'p_NS/MBP100I',
 'p_METO25',
 'p_GLAR100I',
 'p_VIAL',
 'p_AMLO5',
 'p_LORA2I',
 'p_LRLV',
 'p_NS250',
 'p_PNEU25I',
 'p_D545NS1000',
 'p_LR1000',
 'p_FOLI1',
 'p_ASA325',
 'p_MORP2I',
 'p_NS100',
 'p_FURO40I',
 'p_GABA300',
 'p_ACET650L',
 'p_ALBU17H',
 'p_MOM30L',
 'p_HYDR2',
 'p_CLOP75',
 'p_NS451000',
 'p_NEUT',
 'p_PERC',
 'p_PANT40I',
 'p_TRAZ50',
 'p_LORA1',
 'p_TRAM50',
 'p_LISI20',
 'p_DOCU100L',
 'p_RANI150',
 'p_FURO20',
 'p_CIPR500',
 'p_SIME80',
 'p_FRBD50',
 'p_ERGO400T',
 'p_FERR325',
 'p_IPRA2H',
 'p_FU